In [1]:
import cv2 as cv
import numpy as np
import mediapipe as mp
import math

In [2]:
mp_face_mesh = mp.solutions.face_mesh

In [12]:
def getIris(face_landmarks, image):
    img_h, img_w = image.shape[:2]
    mesh_points = np.array([np.multiply([p.x,p.y],[img_w,img_h]).astype(int)for p in face_landmarks[0].landmark])
    (l_cx, l_cy), l_radius = cv.minEnclosingCircle(mesh_points[LEFT_IRIS])
    (r_cx, r_cy), r_radius = cv.minEnclosingCircle(mesh_points[RIGHT_IRIS])
    center_left = np.array([l_cx,l_cy], dtype = np.int32)
    center_right = np.array([r_cx, r_cy], dtype = np.int32)
    llm = mesh_points[R_H_RIGHT][0]
    lrm = mesh_points[R_H_LEFT][0]
    iris_pos,ratio = iris_position(center_right, llm, lrm)
    return iris_pos

In [5]:
RIGHT_IRIS = [474,475,476,477]
LEFT_IRIS = [469,470,471,472]

In [6]:
L_H_LEFT = [33] # right eye right most mark
L_H_RIGHT = [133] # right eye left most mark
R_H_LEFT = [362] #left eye right most mark
R_H_RIGHT = [263] #left eye left most mark

In [8]:
def euclidean_distance(point1, point2):
    x1,y1 = point1.ravel()
    x2,y2 = point2.ravel()
    distance = math.sqrt((x2-x1)**2 + (y2-y1)**2)
    return distance

def iris_position(iris_center, right_point, left_point):
    center_to_right_dist = euclidean_distance(iris_center, right_point)
    total_distance = euclidean_distance(right_point, left_point)
    ratio = center_to_right_dist / total_distance
    iris_position = ""
    if ratio <=0.42:
        iris_position = "right"
    elif 0.42<ratio<=0.57:
        iris_position = 'center'
    else:
        iris_position = 'left'
    return iris_position, ratio

In [14]:
cap = cv.VideoCapture(0)
with mp_face_mesh.FaceMesh(max_num_faces=1, 
                           refine_landmarks=True,
                           min_detection_confidence=0.5,
                           min_tracking_confidence=0.5
                          ) as face_mesh:
    while True:
        ret, frame = cap.read()
        if not ret:
            break
        frame = cv.flip(frame,1)
        rgb_frame = cv.cvtColor(frame, cv.COLOR_BGR2RGB)
        img_h, img_w = frame.shape[:2]
        results = face_mesh.process(rgb_frame)
        if results.multi_face_landmarks:
            iris_pos = getIris(results.multi_face_landmarks, frame)
            cv.putText(frame, f"Eye_Position : {iris_pos}", (10,10), cv.FONT_HERSHEY_PLAIN, 1.2, (0,255,0))
        cv.imshow('img', frame)
        key = cv.waitKey(1)
        if key == ord('q'):
            break
cap.release()
cv.destroyAllWindows()